In [10]:
#importing libraries

import numpy as np 
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests 
from pandas import json_normalize 
import folium 

print('Libraries imported.')

Libraries imported.


In [11]:
CLIENT_ID = 'BIJTHYXEIMVYFPE51M5L33AQMIVUXEKHEP24IAZUSDPDIEIR' # your Foursquare ID
CLIENT_SECRET = 'B04PZEEUF0ZLWXVQJJGFLXCVGFYSYOBN0GGKNSH0ABE5T33J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRECT:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BIJTHYXEIMVYFPE51M5L33AQMIVUXEKHEP24IAZUSDPDIEIR
CLIENT_SECRECT:B04PZEEUF0ZLWXVQJJGFLXCVGFYSYOBN0GGKNSH0ABE5T33J


In [ ]:
LIMIT = 100
radius= 500
cities = ["New York, NY", 'Los Angeles, CA','Washington, D.C.', 'Florida, NY' ,'San Francisco, CA', 'Houston, TX', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        radius,
        LIMIT,
        "4f04af1f2fb6e1c99f3db0bb") # Turkish restaurant CATEGORY ID
    results[city] = requests.get(url).json()

In [32]:
#!pip install ipykernel
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [33]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of turkish restaurant places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of turkish restaurant places in New York, NY =  37
Showing Top 100
Total number of turkish restaurant places in Los Angeles, CA =  3
Showing Top 100
Total number of turkish restaurant places in Washington, D.C. =  6
Showing Top 100
Total number of turkish restaurant places in Florida, NY =  42
Showing Top 100
Total number of turkish restaurant places in San Francisco, CA =  4
Showing Top 100
Total number of turkish restaurant places in Houston, TX =  1
Showing Top 100
Total number of turkish restaurant places in Boston, MA =  3
Showing Top 100


In [34]:
maps[cities[0]]

In [35]:
maps[cities[1]]

In [36]:
maps[cities[2]]

In [37]:
maps[cities[3]]

In [38]:
maps[cities[4]]

In [39]:
maps[cities[5]]

In [40]:
maps[cities[6]]

In [41]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.010244431238725057
Los Angeles, CA
Mean Distance from Mean coordinates
0.00288796285615232
Washington, D.C.
Mean Distance from Mean coordinates
0.0121182683027824
Florida, NY
Mean Distance from Mean coordinates
0.24578387483042302
San Francisco, CA
Mean Distance from Mean coordinates
0.005858591508531393
Houston, TX
Mean Distance from Mean coordinates
0.0
Boston, MA
Mean Distance from Mean coordinates
0.0040439080228325


In [42]:
maps[cities[0]]

In [43]:
maps[cities[1]]

In [44]:
maps[cities[2]]

In [45]:
maps[cities[3]]

In [46]:
maps[cities[4]]

In [47]:
maps[cities[5]]

In [48]:
maps[cities[6]]